![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [66]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import timeit

import warnings

warnings.filterwarnings("ignore")

filename_features = "./Dados/UCI HAR Dataset/features.txt"
filename_labels = "./Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "./Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "./Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "./Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "./Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "./Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "./Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
features = features.squeeze('columns')
feature_names = features.tolist()
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
subject_train=subject_train.squeeze('columns')
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=feature_names)
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
subject_test=subject_test.squeeze('columns')
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=feature_names)
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [46]:
%%time
clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=0.001).fit(X_train, y_train)
train_scores = clf.score(X_train, y_train)
test_scores = clf.score(X_test, y_test)
print(f'A acurácia nas bases de treinamento foi de {train_scores:.4f} e de teste de {test_scores:.4f}')

A acurácia nas bases de treinamento foi de 0.9758 e de teste de 0.8792
CPU times: total: 4.88 s
Wall time: 4.87 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [50]:
%%time
#calcular componente
prcomp = PCA(n_components=1).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

#dataframe
n=1
colunas = ['cp'+str(x+1) for x in list(range(n))]

pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
pc_test  = pd.DataFrame( pc_teste[:,:n], columns = colunas)

#Árvore de decisão
clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=0.001).fit(pc_train, y_train)
train_scores = clf.score(pc_train, y_train)
test_scores = clf.score(pc_test, y_test)
print(f'A acurácia nas bases de treinamento foi de {train_scores:.4f} e de teste de {test_scores:.4f}')

A acurácia nas bases de treinamento foi de 0.4997 e de teste de 0.4571
CPU times: total: 781 ms
Wall time: 235 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [64]:
n = [1, 2, 5, 10, 50]
for i in n:
    start = timeit.default_timer()
    #calcular componente
    prcomp = PCA(n_components=i).fit(X_train)
    
    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)
    
    #dataframe
    n=i
    colunas = ['cp'+str(x+1) for x in list(range(n))]
    
    pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
    pc_test  = pd.DataFrame( pc_teste[:,:n], columns = colunas)
    
    #Árvore de decisão
    clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=0.001).fit(pc_train, y_train)
    stop = timeit.default_timer()
    train_scores = clf.score(pc_train, y_train)
    test_scores = clf.score(pc_test, y_test)
    print(f"Tempo: {stop - start:.2f}s")
    print(f'A acurácia para {i} componetes nas bases de treinamento foi de {train_scores:.4f} e de teste de {test_scores:.4f}')

Tempo: 0.20s
A acurácia para 1 componetes nas bases de treinamento foi de 0.4997 e de teste de 0.4571
Tempo: 0.21s
A acurácia para 2 componetes nas bases de treinamento foi de 0.6128 e de teste de 0.5847
Tempo: 0.21s
A acurácia para 5 componetes nas bases de treinamento foi de 0.8460 e de teste de 0.7886
Tempo: 0.34s
A acurácia para 10 componetes nas bases de treinamento foi de 0.8935 e de teste de 0.8229
Tempo: 0.91s
A acurácia para 50 componetes nas bases de treinamento foi de 0.9166 e de teste de 0.8246


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

    Com a utilização do PCA a tempo reduz drasticamente, porém ao utilizar poucos componentes nota-se que a acurácia diminui significantemente, ao utilizar mais componentes a acurácia aumenta e o tempo também. 